In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
#import catboost as cb
#import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, ShuffleSplit,StratifiedKFold,TimeSeriesSplit,KFold,GroupKFold,train_test_split,GroupShuffleSplit,StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score,mean_squared_error,mean_absolute_error,log_loss,confusion_matrix,accuracy_score
#import sqlite3
#import xgboost as xgb
import datetime
from sklearn.linear_model import LogisticRegression
from scipy.stats import pearsonr
import gc
from sklearn.model_selection import TimeSeriesSplit
#from bayes_opt import BayesianOptimization
import re
from string import punctuation
from scipy.spatial import Voronoi
from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay
from tqdm.notebook import tqdm
#from numba import jit
from collections import Counter
import json
import joblib
import multiprocessing
import time
import keras
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

Using TensorFlow backend.


In [2]:
from sklearn.svm import LinearSVC

In [3]:
### 有click_time
df_bag = pd.read_pickle('../ipynb/df_bag_list_click_time.pkl')

In [4]:
user = pd.read_pickle('../raw_data/user.pkl')

In [5]:
df_bag.head()

,user_id,list_creative_id,list_ad_id,list_advertiser_id,list_product_id
0,1,"[821396, 209778, 877468, 1683713, 122032, 7169...","[724607, 188507, 773445, 1458878, 109959, 6621...","[7293, 9702, 29455, 14668, 11411, 14681, 17189...","[136, 1334, 64, 1454, 64, 1261, 1261]"
1,2,"[63441, 155822, 39714, 609050, 13069, 441462, ...","[58788, 139702, 38066, 541125, 14495, 392680, ...","[22885, 10686, 18562, 25932, 768, 22885, 34505...","[87, 80, 129, 129, 1400, 87, 1261, 111, 129, 1..."
2,3,"[661347, 808612, 710859, 825434, 593522, 72694...","[586668, 713448, 629278, 728308, 527601, 64310...","[32974, 9877, 18492, 14186, 17018, 9058, 8371,...","[36256, 40905, 1674, 35985, 1674, 1031, 1786, ..."
3,4,"[39588, 589886, 574787, 1892854, 31070, 196270...","[37966, 524312, 511235, 1638619, 30773, 169820...","[19451, 7976, 13084, 12130, 13299, 23664, 811,...","[1862, 2625, 38743, 39422, 41265, 37758, 39904..."
4,5,"[296145, 350759, 24333, 43235, 852327, 1054434...","[265971, 314795, 24966, 41148, 751113, 925792,...","[11882, 992, 22885, 9706, 38760, 2862, 2862, 1...","[87, 136, 136, 136, 1064, 2620, 136, 2510, 136..."


#### creative_id

In [6]:
cre_list = df_bag['list_creative_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [7]:
count_vec = CountVectorizer(dtype = np.float32,min_df=50)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [8]:
count_vector_all = count_vec.fit_transform(cre_list)

In [9]:
count_vector_all.shape

(4000000, 269453)

In [10]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.40754666666666667 Cost time: 484
fold_2*********************************************
train go
fold 2 得分：0.4040283333333333 Cost time: 501
fold_3*********************************************
train go
fold 3 得分：0.40708333333333335 Cost time: 488
fold_4*********************************************
train go
fold 4 得分：0.4074583333333333 Cost time: 484
fold_5*********************************************
train go
fold 5 得分：0.40635 Cost time: 440
Mean Accuracy: 0.4064933333333333


In [11]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_count_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_count_pred.pkl')

In [12]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.371835 Cost time: 21
fold_2*********************************************
train go
fold 2 得分：0.369995 Cost time: 23
fold_3*********************************************
train go
fold 3 得分：0.3697683333333333 Cost time: 22
fold_4*********************************************
train go
fold 4 得分：0.37039666666666665 Cost time: 23
fold_5*********************************************
train go
fold 5 得分：0.37102666666666667 Cost time: 22
Mean Accuracy: 0.3706043333333333


In [13]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_count_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_count_pred.pkl')

In [14]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.37766833333333333 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.3772316666666667 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.377175 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.37754166666666666 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.3773916666666667 Cost time: 3
Mean Accuracy: 0.37740166666666664


In [15]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_count_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_count_pred.pkl')

In [16]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.387425 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.387295 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.386585 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.387115 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.3870516666666667 Cost time: 3
Mean Accuracy: 0.3870943333333334


In [17]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_count_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_count_pred.pkl')

In [18]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_base.py:313: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


fold 1 得分：0.3237366666666667 Cost time: 172
fold_2*********************************************
train go
fold 2 得分：0.32781333333333335 Cost time: 171
fold_3*********************************************
train go
fold 3 得分：0.3263 Cost time: 181
fold_4*********************************************
train go
fold 4 得分：0.32781 Cost time: 173
fold_5*********************************************
train go
fold 5 得分：0.32881 Cost time: 177
Mean Accuracy: 0.326894


In [19]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_count_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_count_pred.pkl')

In [20]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = RidgeClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.35623333333333335 Cost time: 2146
fold_2*********************************************
train go
fold 2 得分：0.355315 Cost time: 2289
fold_3*********************************************
train go
fold 3 得分：0.35571166666666665 Cost time: 2264
fold_4*********************************************
train go
fold 4 得分：0.3568366666666667 Cost time: 2275
fold_5*********************************************
train go
fold 5 得分：0.356385 Cost time: 2288
Mean Accuracy: 0.35609633333333335


In [21]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/ridge_count_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/ridge_count_pred.pkl')

#### list_ad_id

In [10]:
cre_list = df_bag['list_ad_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [11]:
count_vec = CountVectorizer(dtype = np.float32,min_df=30)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [12]:
count_vector_all = count_vec.fit_transform(cre_list)

In [13]:
count_vector_all.shape

(4000000, 415926)

In [14]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.4085 Cost time: 526
fold_2*********************************************
train go
fold 2 得分：0.4077033333333333 Cost time: 538
fold_3*********************************************
train go
fold 3 得分：0.41066833333333336 Cost time: 527
fold_4*********************************************
train go
fold 4 得分：0.40872166666666665 Cost time: 535
fold_5*********************************************
train go
fold 5 得分：0.410015 Cost time: 534
Mean Accuracy: 0.4091216666666666


In [15]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_count_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_count_pred_ad.pkl')

In [16]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.37364 Cost time: 24
fold_2*********************************************
train go
fold 2 得分：0.371605 Cost time: 24
fold_3*********************************************
train go
fold 3 得分：0.37153 Cost time: 24
fold_4*********************************************
train go
fold 4 得分：0.37204333333333334 Cost time: 25
fold_5*********************************************
train go
fold 5 得分：0.37237 Cost time: 24
Mean Accuracy: 0.37223766666666663


In [17]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_count_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_count_pred_ad.pkl')

In [18]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.38333 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.38328666666666666 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.383 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.3836733333333333 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.383655 Cost time: 3
Mean Accuracy: 0.38338900000000004


In [19]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_count_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_count_pred_ad.pkl')

In [20]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3918333333333333 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.39192166666666667 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.391085 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.39091333333333333 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.39184833333333335 Cost time: 3
Mean Accuracy: 0.39152033333333336


In [21]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_count_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_count_pred_ad.pkl')

In [22]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_base.py:313: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


fold 1 得分：0.33155833333333334 Cost time: 314
fold_2*********************************************
train go
fold 2 得分：0.33226666666666665 Cost time: 294
fold_3*********************************************
train go
fold 3 得分：0.329205 Cost time: 294
fold_4*********************************************
train go
fold 4 得分：0.3308966666666667 Cost time: 293
fold_5*********************************************
train go
fold 5 得分：0.33182333333333336 Cost time: 299
Mean Accuracy: 0.33115


In [23]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_count_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_count_pred_ad.pkl')

#### list_adv_id

In [24]:
cre_list = df_bag['list_advertiser_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [25]:
count_vec = CountVectorizer(dtype = np.float32,min_df=30)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [26]:
count_vector_all = count_vec.fit_transform(cre_list)

In [27]:
count_vector_all.shape

(4000000, 30113)

In [28]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.37199166666666666 Cost time: 201
fold_2*********************************************
train go
fold 2 得分：0.372595 Cost time: 208
fold_3*********************************************
train go
fold 3 得分：0.3701216666666667 Cost time: 203
fold_4*********************************************
train go
fold 4 得分：0.373635 Cost time: 205
fold_5*********************************************
train go
fold 5 得分：0.37234666666666666 Cost time: 209
Mean Accuracy: 0.37213799999999997


In [29]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_count_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_count_pred_adv.pkl')

In [30]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3680866666666667 Cost time: 24
fold_2*********************************************
train go
fold 2 得分：0.368085 Cost time: 24
fold_3*********************************************
train go
fold 3 得分：0.36628666666666665 Cost time: 24
fold_4*********************************************
train go
fold 4 得分：0.3687566666666667 Cost time: 26
fold_5*********************************************
train go
fold 5 得分：0.36477166666666666 Cost time: 23
Mean Accuracy: 0.3671973333333333


In [31]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_count_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_count_pred_adv.pkl')

In [32]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.33594833333333335 Cost time: 2
fold_2*********************************************
train go
fold 2 得分：0.3363566666666667 Cost time: 2
fold_3*********************************************
train go
fold 3 得分：0.33575666666666665 Cost time: 2
fold_4*********************************************
train go
fold 4 得分：0.337405 Cost time: 2
fold_5*********************************************
train go
fold 5 得分：0.33641 Cost time: 2
Mean Accuracy: 0.33637533333333336


In [33]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_count_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_count_pred_adv.pkl')

In [34]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3551783333333333 Cost time: 2
fold_2*********************************************
train go
fold 2 得分：0.35695333333333334 Cost time: 2
fold_3*********************************************
train go
fold 3 得分：0.35611 Cost time: 2
fold_4*********************************************
train go
fold 4 得分：0.3565833333333333 Cost time: 2
fold_5*********************************************
train go
fold 5 得分：0.35610166666666665 Cost time: 2
Mean Accuracy: 0.35618533333333324


In [35]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_count_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_count_pred_adv.pkl')

In [36]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.2685383333333333 Cost time: 77
fold_2*********************************************
train go


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_base.py:313: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


fold 2 得分：0.28585 Cost time: 79
fold_3*********************************************
train go
fold 3 得分：0.28644 Cost time: 75
fold_4*********************************************
train go
fold 4 得分：0.2748566666666667 Cost time: 79
fold_5*********************************************
train go
fold 5 得分：0.2897516666666667 Cost time: 90
Mean Accuracy: 0.28108733333333336


In [37]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_count_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_count_pred_adv.pkl')

#### list_prod_id

In [38]:
cre_list = df_bag['list_product_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [39]:
count_vec = CountVectorizer(dtype = np.float32,min_df=30)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [40]:
count_vector_all = count_vec.fit_transform(cre_list)

In [41]:
count_vector_all.shape

(4000000, 14203)

In [42]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.291315 Cost time: 163
fold_2*********************************************
train go
fold 2 得分：0.29275 Cost time: 160
fold_3*********************************************
train go
fold 3 得分：0.29241 Cost time: 160
fold_4*********************************************
train go
fold 4 得分：0.29373 Cost time: 161
fold_5*********************************************
train go
fold 5 得分：0.2908616666666667 Cost time: 161
Mean Accuracy: 0.2922133333333334


In [43]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_count_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_count_pred_prod.pkl')

In [44]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.29054833333333335 Cost time: 21
fold_2*********************************************
train go
fold 2 得分：0.2923 Cost time: 28
fold_3*********************************************
train go
fold 3 得分：0.2872133333333333 Cost time: 21
fold_4*********************************************
train go
fold 4 得分：0.2908766666666667 Cost time: 24
fold_5*********************************************
train go
fold 5 得分：0.28815666666666667 Cost time: 21
Mean Accuracy: 0.289819


In [45]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_count_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_count_pred_prod.pkl')

In [46]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.260585 Cost time: 1
fold_2*********************************************
train go
fold 2 得分：0.26074833333333336 Cost time: 1
fold_3*********************************************
train go
fold 3 得分：0.259875 Cost time: 1
fold_4*********************************************
train go
fold 4 得分：0.2615766666666667 Cost time: 1
fold_5*********************************************
train go
fold 5 得分：0.260605 Cost time: 1
Mean Accuracy: 0.2606780000000001


In [47]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_count_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_count_pred_prod.pkl')

In [48]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.2841266666666667 Cost time: 1
fold_2*********************************************
train go
fold 2 得分：0.28441833333333333 Cost time: 1
fold_3*********************************************
train go
fold 3 得分：0.28264333333333336 Cost time: 1
fold_4*********************************************
train go
fold 4 得分：0.28433166666666665 Cost time: 1
fold_5*********************************************
train go
fold 5 得分：0.2832983333333333 Cost time: 1
Mean Accuracy: 0.28376366666666664


In [49]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_count_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_count_pred_prod.pkl')

In [50]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.21631666666666666 Cost time: 56
fold_2*********************************************
train go
fold 2 得分：0.226775 Cost time: 62
fold_3*********************************************
train go
fold 3 得分：0.20886166666666667 Cost time: 70
fold_4*********************************************
train go


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_base.py:313: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


fold 4 得分：0.21067 Cost time: 56
fold_5*********************************************
train go
fold 5 得分：0.19653833333333334 Cost time: 52
Mean Accuracy: 0.21183233333333334


In [51]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_count_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_count_pred_prod.pkl')

### tfidf

#### creative_id

In [52]:
cre_list = df_bag['list_creative_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [53]:
count_vec = TfidfVectorizer(dtype = np.float32,min_df=30)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [54]:
count_vector_all = count_vec.fit_transform(cre_list)

In [55]:
count_vector_all.shape

(4000000, 416682)

In [56]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3700833333333333 Cost time: 528
fold_2*********************************************
train go
fold 2 得分：0.36757 Cost time: 584
fold_3*********************************************
train go
fold 3 得分：0.3670616666666667 Cost time: 601
fold_4*********************************************
train go
fold 4 得分：0.36748166666666665 Cost time: 584
fold_5*********************************************
train go
fold 5 得分：0.367625 Cost time: 609
Mean Accuracy: 0.3679643333333333


In [57]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_tfidf_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_tfidf_pred.pkl')

In [58]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.256275 Cost time: 20
fold_2*********************************************
train go
fold 2 得分：0.25498666666666664 Cost time: 19
fold_3*********************************************
train go
fold 3 得分：0.255035 Cost time: 19
fold_4*********************************************
train go
fold 4 得分：0.2552833333333333 Cost time: 19
fold_5*********************************************
train go
fold 5 得分：0.2552316666666667 Cost time: 20
Mean Accuracy: 0.25536233333333336


In [59]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_tfidf_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_tfidf_pred.pkl')

In [60]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.38745666666666667 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.3868916666666667 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.38634833333333335 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.3872333333333333 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.38626666666666665 Cost time: 3
Mean Accuracy: 0.38683933333333337


In [61]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_tfidf_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_tfidf_pred.pkl')

In [62]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.39012166666666664 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.39077 Cost time: 4
fold_3*********************************************
train go
fold 3 得分：0.389895 Cost time: 4
fold_4*********************************************
train go
fold 4 得分：0.39023166666666664 Cost time: 4
fold_5*********************************************
train go
fold 5 得分：0.3902233333333333 Cost time: 4
Mean Accuracy: 0.3902483333333333


In [63]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_tfidf_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_tfidf_pred.pkl')

In [64]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3486916666666667 Cost time: 242
fold_2*********************************************
train go
fold 2 得分：0.34913333333333335 Cost time: 251
fold_3*********************************************
train go
fold 3 得分：0.35141333333333336 Cost time: 246
fold_4*********************************************
train go
fold 4 得分：0.34980666666666665 Cost time: 245
fold_5*********************************************
train go
fold 5 得分：0.351 Cost time: 240
Mean Accuracy: 0.35000899999999996


In [65]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_tfidf_oof.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_tfidf_pred.pkl')

#### list_ad_id

In [66]:
cre_list = df_bag['list_ad_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [67]:
count_vec = CountVectorizer(dtype = np.float32,min_df=30)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [68]:
count_vector_all = count_vec.fit_transform(cre_list)

In [69]:
count_vector_all.shape

(4000000, 415926)

In [70]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.4085 Cost time: 526
fold_2*********************************************
train go
fold 2 得分：0.4077033333333333 Cost time: 555
fold_3*********************************************
train go
fold 3 得分：0.41066833333333336 Cost time: 545
fold_4*********************************************
train go
fold 4 得分：0.40872166666666665 Cost time: 563
fold_5*********************************************
train go
fold 5 得分：0.410015 Cost time: 541
Mean Accuracy: 0.4091216666666666


In [71]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_tfidf_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_tfidf_pred_ad.pkl')

In [72]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.37364 Cost time: 23
fold_2*********************************************
train go
fold 2 得分：0.371605 Cost time: 24
fold_3*********************************************
train go
fold 3 得分：0.37153 Cost time: 24
fold_4*********************************************
train go
fold 4 得分：0.37204333333333334 Cost time: 23
fold_5*********************************************
train go
fold 5 得分：0.37237 Cost time: 24
Mean Accuracy: 0.37223766666666663


In [73]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_tfidf_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_tfidf_pred_ad.pkl')

In [74]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.38333 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.38328666666666666 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.383 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.3836733333333333 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.383655 Cost time: 3
Mean Accuracy: 0.38338900000000004


In [75]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_tfidf_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_tfidf_pred_ad.pkl')

In [76]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3918333333333333 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.39192166666666667 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.391085 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.39091333333333333 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.39184833333333335 Cost time: 3
Mean Accuracy: 0.39152033333333336


In [77]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_tfidf_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_tfidf_pred_ad.pkl')

In [78]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_base.py:313: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


fold 1 得分：0.33155833333333334 Cost time: 311
fold_2*********************************************
train go
fold 2 得分：0.33226666666666665 Cost time: 292
fold_3*********************************************
train go
fold 3 得分：0.329205 Cost time: 292
fold_4*********************************************
train go
fold 4 得分：0.3308966666666667 Cost time: 289
fold_5*********************************************
train go
fold 5 得分：0.33182333333333336 Cost time: 295
Mean Accuracy: 0.33115


In [79]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_tfidf_oof_ad.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_tfidf_pred_ad.pkl')

#### list_adv_id

In [80]:
cre_list = df_bag['list_advertiser_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [81]:
count_vec = CountVectorizer(dtype = np.float32,min_df=30)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [82]:
count_vector_all = count_vec.fit_transform(cre_list)

In [83]:
count_vector_all.shape

(4000000, 30113)

In [84]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.37199166666666666 Cost time: 215
fold_2*********************************************
train go
fold 2 得分：0.372595 Cost time: 224
fold_3*********************************************
train go
fold 3 得分：0.3701216666666667 Cost time: 230
fold_4*********************************************
train go
fold 4 得分：0.373635 Cost time: 223
fold_5*********************************************
train go
fold 5 得分：0.37234666666666666 Cost time: 232
Mean Accuracy: 0.37213799999999997


In [85]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_tfidf_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_tfidf_pred_adv.pkl')

In [86]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3680866666666667 Cost time: 24
fold_2*********************************************
train go
fold 2 得分：0.368085 Cost time: 23
fold_3*********************************************
train go
fold 3 得分：0.36628666666666665 Cost time: 24
fold_4*********************************************
train go
fold 4 得分：0.3687566666666667 Cost time: 26
fold_5*********************************************
train go
fold 5 得分：0.36477166666666666 Cost time: 23
Mean Accuracy: 0.3671973333333333


In [87]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_tfidf_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_tfidf_pred_adv.pkl')

In [88]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.33594833333333335 Cost time: 2
fold_2*********************************************
train go
fold 2 得分：0.3363566666666667 Cost time: 2
fold_3*********************************************
train go
fold 3 得分：0.33575666666666665 Cost time: 2
fold_4*********************************************
train go
fold 4 得分：0.337405 Cost time: 2
fold_5*********************************************
train go
fold 5 得分：0.33641 Cost time: 2
Mean Accuracy: 0.33637533333333336


In [89]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_tfidf_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_tfidf_pred_adv.pkl')

In [90]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.3551783333333333 Cost time: 3
fold_2*********************************************
train go
fold 2 得分：0.35695333333333334 Cost time: 3
fold_3*********************************************
train go
fold 3 得分：0.35611 Cost time: 3
fold_4*********************************************
train go
fold 4 得分：0.3565833333333333 Cost time: 3
fold_5*********************************************
train go
fold 5 得分：0.35610166666666665 Cost time: 3
Mean Accuracy: 0.35618533333333324


In [91]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_tfidf_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_tfidf_pred_adv.pkl')

In [92]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.2685383333333333 Cost time: 77
fold_2*********************************************
train go


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_base.py:313: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


fold 2 得分：0.28585 Cost time: 90
fold_3*********************************************
train go
fold 3 得分：0.28644 Cost time: 86
fold_4*********************************************
train go
fold 4 得分：0.2748566666666667 Cost time: 80
fold_5*********************************************
train go
fold 5 得分：0.2897516666666667 Cost time: 94
Mean Accuracy: 0.28108733333333336


In [93]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_tfidf_oof_adv.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_tfidf_pred_adv.pkl')

#### list_prod_id

In [94]:
cre_list = df_bag['list_product_id'].apply(lambda x: str(x).replace('[','').replace(']','').replace(',',' '))

In [95]:
count_vec = CountVectorizer(dtype = np.float32,min_df=30)
#tfidf_vec = TfidfVectorizer(dtype = np.float32,min_df=30)

In [96]:
count_vector_all = count_vec.fit_transform(cre_list)

In [97]:
count_vector_all.shape

(4000000, 14203)

In [98]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = LogisticRegression(random_state=47,n_jobs=8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.291315 Cost time: 162
fold_2*********************************************
train go
fold 2 得分：0.29275 Cost time: 159
fold_3*********************************************
train go
fold 3 得分：0.29241 Cost time: 158
fold_4*********************************************
train go
fold 4 得分：0.29373 Cost time: 160
fold_5*********************************************
train go
fold 5 得分：0.2908616666666667 Cost time: 160
Mean Accuracy: 0.2922133333333334


In [99]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/logistic_tfidf_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/logistic_tfidf_pred_prod.pkl')

In [100]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = SGDClassifier(random_state=47,loss='log',n_jobs = 8)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.29054833333333335 Cost time: 21
fold_2*********************************************
train go
fold 2 得分：0.2923 Cost time: 27
fold_3*********************************************
train go
fold 3 得分：0.2872133333333333 Cost time: 22
fold_4*********************************************
train go
fold 4 得分：0.2908766666666667 Cost time: 24
fold_5*********************************************
train go
fold 5 得分：0.28815666666666667 Cost time: 21
Mean Accuracy: 0.289819


In [101]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/SGDClassifier_tfidf_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/SGDClassifier_tfidf_pred_prod.pkl')

In [102]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = MultinomialNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.260585 Cost time: 1
fold_2*********************************************
train go
fold 2 得分：0.26074833333333336 Cost time: 1
fold_3*********************************************
train go
fold 3 得分：0.259875 Cost time: 1
fold_4*********************************************
train go
fold 4 得分：0.2615766666666667 Cost time: 1
fold_5*********************************************
train go
fold 5 得分：0.260605 Cost time: 1
Mean Accuracy: 0.2606780000000001


In [103]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/multinNB_tfidf_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/multinNB_tfidf_pred_prod.pkl')

In [104]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = BernoulliNB()
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf.predict_proba(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf.predict_proba(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.2841266666666667 Cost time: 1
fold_2*********************************************
train go
fold 2 得分：0.28441833333333333 Cost time: 1
fold_3*********************************************
train go
fold 3 得分：0.28264333333333336 Cost time: 1
fold_4*********************************************
train go
fold 4 得分：0.28433166666666665 Cost time: 1
fold_5*********************************************
train go
fold 5 得分：0.2832983333333333 Cost time: 1
Mean Accuracy: 0.28376366666666664


In [105]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/BernoulliNB_tfidf_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/BernoulliNB_tfidf_pred_prod.pkl')

In [106]:
kf = KFold(n_splits=5,shuffle=True,random_state=47)

y_train_age = user['age']
y_train_gender = user['gender']

X_train = count_vector_all[:3000000]
X_test = count_vector_all[3000000:]

k = 1
pred_gender = 0
y_train_pred = np.zeros((3000000,10))
score_list = []
for train_index,test_index in kf.split(X_train,y_train_age):
    
    start = time.time()
    print(f'fold_{k}*********************************************')
    k+=1
    X_train2 = X_train[train_index,:]
    y_train2 = y_train_age.iloc[train_index]
    X_test2 = X_train[test_index,:]
    y_test2 = y_train_age.iloc[test_index]
    
    clf = PassiveAggressiveClassifier(random_state=1017)
    print('train go')
    clf.fit(X_train2,y_train2)
    end = time.time()
    
    y_train_pred[test_index] = clf._predict_proba_lr(X_test2)
    tmp_score = accuracy_score(y_test2,clf.predict(X_test2))
    pred_gender += clf._predict_proba_lr(X_test)/kf.n_splits
    
    score_list.append(tmp_score)
    print(f'fold {k-1} 得分：{tmp_score}','Cost time:',round(end-start))
    gc.collect()
print('Mean Accuracy:',np.mean(score_list))

fold_1*********************************************
train go
fold 1 得分：0.21631666666666666 Cost time: 56
fold_2*********************************************
train go
fold 2 得分：0.226775 Cost time: 63
fold_3*********************************************
train go
fold 3 得分：0.20886166666666667 Cost time: 72
fold_4*********************************************
train go


C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\linear_model\_base.py:313: RuntimeWarning: invalid value encountered in true_divide
  prob /= prob.sum(axis=1).reshape((prob.shape[0], -1))


fold 4 得分：0.21067 Cost time: 57
fold_5*********************************************
train go
fold 5 得分：0.19653833333333334 Cost time: 55
Mean Accuracy: 0.21183233333333334


In [107]:
pd.DataFrame(y_train_pred).to_pickle('tf-idf-count/pac_tfidf_oof_prod.pkl')
pd.DataFrame(pred_gender).to_pickle('tf-idf-count/pac_tfidf_pred_prod.pkl')